# Organización del suelo industrial

## Limpieza de los datos

<b>Objetivo:</b> eliminar las líneas innecesarias, y organizar los encabezados de los documentos. De esta manera podremos unificar todos los datos posteriormente.

In [1]:
import os

def clean_data(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    csv_files = [file for file in os.listdir(input_folder) if file.endswith('.csv')]

    # Datos excepcionales
    unuseful_words = ['actividad', 'Serie;;;']

    # Limpiar y guardar los archivos CSV en la carpeta de salida
    for file in csv_files:
        with open(os.path.join(input_folder, file), 'r') as f_input, open(os.path.join(output_folder, file), 'w') as f_output:
            delete_next = False
            for line in f_input:                    
                if not delete_next:
                    delete = False
                    for word in unuseful_words:
                        if word in line:
                            delete = True
                    if line.strip() == '':
                        delete = True
                    elif '-' in line:
                        line = line.replace('-', '0')

                    # Eliminar información insuficiente
                    fields = line.strip().split(';')
                    if len(fields) <= 12 or not fields[3].isdigit():
                        delete = True
                    if not delete:
                        f_output.write(';'.join([str(x) for x in [fields[4], fields[3], fields[9]] + fields[11:13]]) + '\n')
                    if 'Zarzalejo' in line:
                        delete_next = True

input_folder = ".\original_data_parcelas"
output_folder = ".\clean_data_parcelas"

clean_data(input_folder, output_folder)


### Unión de los datos de las parcelas

<b>Objetivo:</b> unir los datos de superficie edificable de cada municipio y exportarlo como un nuevo documento. De esta manera podemos diferenciar entre uso industrial parcelario y uso terciario industrial.

In [2]:
import csv

def join_data(input_file, output_file):

    # Diccionarios para almacenar la suma de superficies edificables y totales por municipio y uso principal
    superficie_edificable = {}
    superficie_total = {}

    with open(input_file, mode='r') as file:
        reader = csv.reader(file, delimiter=';')
        next(reader)
        
        # Iterar sobre las filas del archivo de entrada
        for row in reader:
            municipio = row[0]
            codigo_municipio = row[1]
            uso_principal = row[2]
            # Convertir la superficie edificable a tipo float, con manejo de errores
            try:
                superficie_edif = float(row[3].replace(',', '.'))
            except ValueError:
                superficie_edif = 0.0
            
            # Convertir la superficie total a tipo float, con manejo de errores
            try:
                superficie_total_actual = float(row[4].replace(',', '.')) if row[4] else 0.0
            except ValueError:
                superficie_total_actual = 0.0
            
            # Actualizar la suma de superficies edificables y totales para el municipio y uso principal correspondientes
            if (municipio, codigo_municipio, uso_principal) in superficie_edificable:
                superficie_edificable[(municipio, codigo_municipio, uso_principal)] += superficie_edif
                superficie_total[(municipio, codigo_municipio, uso_principal)] += superficie_total_actual
            else:
                superficie_edificable[(municipio, codigo_municipio, uso_principal)] = superficie_edif
                superficie_total[(municipio, codigo_municipio, uso_principal)] = superficie_total_actual

    # Crear un diccionario para almacenar los datos combinados
    join_data = {}

    # Iterar sobre los datos de uso industrial
    for (municipio, codigo_municipio, uso_principal), superficie_edif in superficie_edificable.items():
        if uso_principal == 'T' or uso_principal == 'I':
            key = (municipio, codigo_municipio)
            if key not in join_data:
                join_data[key] = {'Municipio': municipio, 'codigo_municipio': codigo_municipio}
            if uso_principal == 'I':
                join_data[key]['superficie_edificable_industrial'] = round(superficie_edif, 2)
                join_data[key]['superficie_total_industrial'] = round(superficie_total[(municipio, codigo_municipio, uso_principal)], 2)
            elif uso_principal == 'T':
                join_data[key]['superficie_edificable_terciario_industrial'] = round(superficie_edif, 2)
                join_data[key]['superficie_total_terciario_industrial'] = round(superficie_total[(municipio, codigo_municipio, uso_principal)], 2)

    # Crear un diccionario para almacenar los datos combinados
    join_data = {}

    # Iterar sobre los datos de uso industrial
    for (municipio, codigo_municipio, uso_principal), superficie_edif in superficie_edificable.items():
        if uso_principal == 'T' or uso_principal == 'I': # Solo uso industrial y terciario industrial
            key = (municipio, codigo_municipio)
            if key not in join_data:
                join_data[key] = {'Municipio': municipio, 'codigo_municipio': codigo_municipio}
            if uso_principal == 'I':
                join_data[key]['sup_edif_indust'] = round(superficie_edif, 2)
                join_data[key]['sup_total_indust'] = round(superficie_total[(municipio, codigo_municipio, uso_principal)], 2)
            elif uso_principal == 'T':
                join_data[key]['sup_edif_terc_indust'] = round(superficie_edif, 2)
                join_data[key]['sup_total_terc_indust'] = round(superficie_total[(municipio, codigo_municipio, uso_principal)], 2)

    # Escribir los datos combinados en un nuevo archivo CSV
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, delimiter=';', fieldnames=['Municipio', 'codigo_municipio', 
                                                                'sup_edif_indust', 'sup_total_indust', 
                                                                'sup_edif_terc_indust', 'sup_total_terc_indust'])
        writer.writeheader()
        for municipio_data in join_data.values():
            # Completar los campos faltantes con cadena vacía
            municipio_data.setdefault('sup_edif_indust', '')
            municipio_data.setdefault('sup_total_indust', '')
            municipio_data.setdefault('sup_edif_terc_indust', '')
            municipio_data.setdefault('sup_total_terc_indust', '')
            writer.writerow(municipio_data)


    input_file = './clean_data_parcelas/parcelas_portal_suelo.csv'
    output_file = './parcelas_industriales_sin_formato.csv'

    join_data(input_file, output_file)

### Regulación de formato de las parcelas

<b>Objetivo:</b> regular el formato del archivo al mismo que los demás, incluyendo los municipios que no tienen parcelas industriales. Para ello cogemos un archivo al azar, como puede ser <i>paro.csv</i>

In [3]:
import csv

def refactor_file(example_file, input_file, output_file):
    # Diccionario para almacenar los datos del primer archivo
    data = {}

    # Leer el primer archivo y almacenar los datos relevantes en el diccionario
    with open(example_file, mode='r') as f_base:
        base_reader = csv.reader(f_base, delimiter=';')
        headers_base = next(base_reader)[:3]

        # Crear una clave única para cada entrada
        for row in base_reader:
            key = ';'.join(row[:3])  
            data[key] = {}

    # Leer el segundo archivo y combinar los datos
    with open(input_file, mode='r') as f2:
        industrial_reader = csv.reader(f2, delimiter=';')
        rest_headers = next(industrial_reader)[2:]

        for row in industrial_reader:
            nombre = row[0]  # Crear una clave única para cada entrada
            for key in data:
                if nombre in key:
                    data[key] = row[2:]

    # Escribir los datos sumados en un nuevo archivo CSV
    with open(output_file, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')
        
        # Escribir los encabezados
        headers = headers_base + rest_headers
        csv_writer.writerow(headers)

        # Escribir los datos sumados
        for key, values in data.items():
            line = [key.split(';')[0], key.split(';')[1], key.split(';')[2]] + [v if v != '' else '0' for v in values]
            
            # Añadir valores nulos para que todas las filas tengan el mismo número de datos
            while len(line) < len(headers):
                line.append('0')
            
            csv_writer.writerow(line)

example_file = '../union_final/paro.csv'
input_file = 'parcelas_industriales_sin_formato.csv'
output_file = '../union_final/parcelas_industriales.csv'

refactor_file(example_file, input_file, output_file)